In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import random
from geopy.distance import geodesic
from scipy.optimize import linear_sum_assignment

In [3]:
!pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 4.4 MB/s eta 0:00:00


In [60]:
weight_data = pd.read_csv('/content/drive/MyDrive/final project/קודים גמורים/Data/weight.csv')
clean_data_df = pd.read_csv('/content/drive/MyDrive/final project/קודים גמורים/Data/clean_data.csv')
weight_data.head()

<ipython-input-60-e0ba303ca06c>:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  clean_data_df = pd.read_csv('/content/drive/MyDrive/final project/קודים גמורים/Data/clean_data.csv')
/usr/local/lib/python3.10/dist-packages/pandas/core/nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,מזהה כונן,קטגוריה,ממוצע זמן טיפול בתקלה,כמות אירועים,אירועים מוצלחים,אחוזי הצלחה,ממוצע דירוג לקוח,t_ratio,r_ratio,w_ratio,t_score,r_score,w_score,rank,final score
0,0001 שלמה,אחר,24.000000,1,1.0,100.0,2.944444,0.975470,0.980425,1.159445,3,3,4,2.356700,10.182565
1,0003 שלומי,אחר,14.000000,1,1.0,100.0,3.250000,1.672234,1.082168,1.159445,5,4,4,3.119162,14.574226
2,0004 אליהו,אחר,1.000000,1,1.0,100.0,3.454545,23.411283,1.150276,1.159445,5,4,4,3.119162,14.574226
3,0009 דדי,אחר,25.280388,1,1.0,100.0,1.000000,0.926065,0.332975,1.159445,3,1,4,2.218071,9.384081
4,0009 יוסי,אחר,12.640194,6,5.0,100.0,2.869565,1.852130,0.955493,1.159445,5,3,4,8.562005,45.924147


סינון קטגוריות לא רלוונטיות

In [61]:
categories=['הנעה', "פנצ'ר", 'רכב נעול','דלת', 'אחר','שמן-מים-דלק', 'שינוע', 'חילוץ שטח']

# סינון השורות בהן הערכים בעמודת 'קטגוריה' נמצאים ברשימת הקטגוריות
clean_data_df = clean_data_df[clean_data_df['קטגוריה'].isin(categories)]

clean_data_df


,זמן,כתובת,רכב,קטגוריה,תת קטגוריה,מזהה פונה,מזהה כונן,זמן לקיחה,זמן סגירה,סטטוס אירוע,זמן טיפול,זמן טיפול ממוצע,חריג זמן טיפול,קו רוחב תקלה,קו אורך תקלה,קו רוחב מתנדב,קו אורך מתנדב,"מרחק בק""מ",דירוג לקוח
0,2022-01-01 17:39:00,"הנביאים 56, בת ים, ישראל",מרצדס ויטו,הנעה,כבלים חזקים,9043 ניסים,1123 גיל,2022-01-01 17:39:00.000000000,2022-01-01 18:16:00.000000000,נסגר בהצלחה,37.000000,18.064785,False,32.739057,35.236570,32.764670,35.227373,2.968398,2.0
1,2022-01-01 17:39:00,"קדיש לוז 22, קרית מוצקין, ישראל",קיה קארנס,פנצ'ר,פנצ'ר + כלים,0574 אבי,4278 שלמה,2022-01-01 18:27:00.000000000,2022-01-01 18:37:00.000000000,נסגר בהצלחה,10.000000,25.305340,False,30.218503,34.374803,30.185443,34.399038,4.344723,2.0
2,2022-01-01 17:39:00,"דקר 21, לוד, ישראל",מיצובישי אאוטלנדר,פנצ'ר,פנצ'ר + כלים,7308 שמעון,4838 משה,2022-01-01 18:28:00.000000000,2022-01-01 18:53:17.501820218,נסגר בהצלחה,25.291697,25.305340,False,32.715484,35.275912,32.704803,35.286877,1.568408,1.0
3,2022-01-01 17:41:00,"אבנר בן יהודה 31, נס ציונה, ישראל",טויוטה קורולה,פנצ'ר,פנצ'ר,2629 שמרית,8697 גרשון,2022-01-01 19:51:00.000000000,2022-01-01 20:16:17.501820218,נסגר בהצלחה,25.291697,25.305340,False,33.164280,35.110036,33.142977,35.154884,4.804959,3.0
4,2022-01-01 17:43:00,"תחנת עוז נתניה, הרצל 102, נתניה, ישראל",סוזוקי אלטו,הנעה,כבלים,0363 יגאל,1940 אורי,2022-01-01 17:56:00.000000000,2022-01-01 18:06:00.000000000,נסגר בהצלחה,10.000000,18.064785,False,30.135537,34.541962,30.162403,34.536735,3.020540,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307522,2022-12-31 23:57:00,"סולם יעקב & רח' רָמוֹת, ירושלים",שברולט טאהו / יוקון,הנעה,כבלים,1000 שמאל,1595 דניאל,2022-01-01 00:00:00,2022-01-01 00:18:02.799164210,נסגר בהצלחה,18.046653,18.064785,False,29.659043,35.432397,29.669615,35.397475,3.578130,4.0
307523,2022-12-31 23:57:00,"כינרת 3, רמלה, ישראל",קיה סורנטו,אחר,אחר,1037 שלמה,6737 משה,2022-01-01 09:01:00,2022-01-01 09:26:16.823256793,נסגר בהצלחה,25.280388,25.280388,False,32.747308,34.378830,32.700671,34.382908,5.186169,1.0
307524,2022-12-31 23:58:00,"העבודה 3, אריאל",סקודה קודיאק,פנצ'ר,פנצ'ר,8573 אפרת,1643 קובי,2022-01-01 00:46:00,2022-01-01 00:51:00.000000000,נסגר בהצלחה,5.000000,25.305340,False,30.409886,35.551801,30.399730,35.544906,1.306448,1.0
307525,2022-12-31 23:59:00,"הרצל 112, רחובות, ישראל",פולקסווגן ג'יפ טוארג,הנעה,כבלים חזקים,6461 אושר,1674 שמעון,2022-01-01 00:00:00,2022-01-01 00:40:00.000000000,נסגר בהצלחה,40.000000,18.064785,False,32.238833,35.317162,32.236280,35.365303,4.545994,3.0


In [62]:
clean_data_df['קטגוריה'].unique()

array(['הנעה', "פנצ'ר", 'רכב נעול', 'דלת', 'אחר', 'שמן-מים-דלק', 'שינוע',
       'חילוץ שטח'], dtype=object)

In [9]:


# מחיקת עמודות לא רלוונטיות
columns_to_keep = ['מזהה כונן','קטגוריה','final score']
Volunteer_pool = weight_data[columns_to_keep]

# יצירת עמודה רנדומלית 'זמינות' עם ערכים 'זמין' או 'לא זמין'
Volunteer_pool['זמינות'] = [random.choice(['זמין', 'לא זמין']) for _ in range(len(Volunteer_pool))]

# פונקציה ליצירת קורדינטות רנדומליות בשטחי ישראל
def random_coordinates_in_israel():
    lat = random.uniform(29.5, 33.3)  # תחום קווי רוחב של ישראל
    lon = random.uniform(34.2, 35.9)  # תחום קווי אורך של ישראל
    return f"{lat},{lon}"

# הוספת עמודת 'מיקום כונן' על פי העמודה 'זמינות'
Volunteer_pool['מיקום כונן'] = Volunteer_pool['זמינות'].apply(
    lambda x: random_coordinates_in_israel() if x == 'זמין' else None
)

# הצגת התוצאות האחרונות
Volunteer_pool.tail(20)


<ipython-input-9-6710bd232c48>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Volunteer_pool['זמינות'] = [random.choice(['זמין', 'לא זמין']) for _ in range(len(Volunteer_pool))]
<ipython-input-9-6710bd232c48>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Volunteer_pool['מיקום כונן'] = Volunteer_pool['זמינות'].apply(


,מזהה כונן,קטגוריה,final score,זמינות,מיקום כונן
21784,9930 דוד,שמן-מים-דלק,7.285183,זמין,"29.683540432429037,35.78966651084123"
21785,9930 עדי,שמן-מים-דלק,17.855717,זמין,"33.144204748981366,34.404856430047204"
21786,9933 יהודה,שמן-מים-דלק,10.142084,זמין,"31.873683532638907,34.4035779547188"
21787,9936 עמרם,שמן-מים-דלק,17.284337,לא זמין,None
21788,9941 בנצי,שמן-מים-דלק,8.713633,לא זמין,None
21789,9945 לאון,שמן-מים-דלק,7.570873,לא זמין,None
21790,9949 איתן,שמן-מים-דלק,13.705693,לא זמין,None
21791,9950 אור,שמן-מים-דלק,21.074726,זמין,"32.16314504344126,35.1396630590011"
21792,9953 אבי,שמן-מים-דלק,20.712618,זמין,"31.784782606245358,34.6047559839392"
21793,9957 מרדכי,שמן-מים-דלק,10.427774,לא זמין,None


In [40]:
# Volunteer_pool.to_csv('volunteer_pool.csv', index=False)

In [22]:
# מחיקת עמודות לא רלוונטיות
columns_to_keep = ['קטגוריה', 'מזהה פונה']
calling_pool = clean_data_df[columns_to_keep]

# פונקציה ליצירת קורדינטות רנדומליות בשטחי ישראל
def random_coordinates_in_israel():
    lat = random.uniform(29.5, 33.3)  # תחום קווי רוחב של ישראל
    lon = random.uniform(34.2, 35.9)  # תחום קווי אורך של ישראל
    return f"{lat},{lon}"

# הוספת עמודה 'מיקום תקלה' עם קורדינטות רנדומליות
calling_pool['מיקום תקלה'] = [random_coordinates_in_israel() for _ in range(len(calling_pool))]

# הוספת עמודת 'מס' קריאה' עם מספר רנדומלי בן 5 ספרות
calling_pool['מספר קריאה'] = [random.randint(10000, 99999) for _ in range(len(calling_pool))]
calling_pool

<ipython-input-22-976205ca5c83>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calling_pool['מיקום תקלה'] = [random_coordinates_in_israel() for _ in range(len(calling_pool))]
<ipython-input-22-976205ca5c83>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calling_pool['מספר קריאה'] = [random.randint(10000, 99999) for _ in range(len(calling_pool))]


,קטגוריה,מזהה פונה,מיקום תקלה,מספר קריאה
0,הנעה,9043 ניסים,"32.700882245453506,35.093420563525186",63334
1,פנצ'ר,0574 אבי,"30.423805731885466,35.53096361286402",17293
2,פנצ'ר,7308 שמעון,"31.28589510337637,34.70028550345861",62679
3,פנצ'ר,2629 שמרית,"31.569033554726825,35.516761250878176",43837
4,הנעה,0363 יגאל,"30.72311782323087,34.513752163275875",36610
...,...,...,...,...
307522,הנעה,1000 שמאל,"32.54408455299488,35.530553441519736",67741
307523,אחר,1037 שלמה,"32.58356994929611,35.70489920018762",63844
307524,פנצ'ר,8573 אפרת,"29.695925462066977,34.76667164071631",10595
307525,הנעה,6461 אושר,"29.839427412095432,35.60736855207088",31125


In [11]:
# פונקציה לנרמול ציונים
def normalize_scores(scores):
    max_score = max(scores)
    return max_score - scores

# שלב 1: הגרלת מספר רנדומלי בין 1 ל-20
num_rows = random.randint(1, 20)

# שלב 2: הגרלת מספר רנדומלי של שורות מהטבלה calling_pool
random_rows = calling_pool.sample(n=num_rows)

# שלב 3: פיצול הטבלה calling_pool לפי קטגוריות
calling_by_category = random_rows.groupby('קטגוריה')

# שלב 4: סינון עמודת זמינות בטבלה volunteer_pool
available_volunteers = Volunteer_pool[Volunteer_pool['זמינות'] == 'זמין']

# שלב 5: פיצול הטבלה volunteer_pool לפי קטגוריות
volunteer_by_category = available_volunteers.groupby('קטגוריה')

# שלב 6: חישוב מרחקים בין המיקומים
results = []

for category, call_df in calling_by_category:
    if category not in volunteer_by_category.groups:
        continue

    vol_df = volunteer_by_category.get_group(category)

    call_df.loc[:, 'מיקום תקלה'] = call_df['מיקום תקלה'].apply(eval)
    vol_df.loc[:, 'מיקום כונן'] = vol_df['מיקום כונן'].apply(eval)

    # הכנת מטריצת עלויות (מרחק + ציון מתנדב מנורמל)
    cost_matrix = []
    for call_index, call_row in call_df.iterrows():
        row = []
        call_location = call_row['מיקום תקלה']
        for vol_index, vol_row in vol_df.iterrows():
            vol_location = vol_row['מיקום כונן']
            distance = geodesic(call_location, vol_location).kilometers
            row.append(distance)
        cost_matrix.append(row)

    cost_matrix = np.array(cost_matrix)

    # נרמול ציוני המתנדבים
    volunteer_scores = normalize_scores(vol_df['final score'])

    # הוספת הציונים למטריצת העלויות
    cost_matrix += volunteer_scores.values.reshape(1, -1)

    # וידוא שהמטריצה היא ריבועית על ידי הוספת שורות/עמודות עם עלויות גבוהות מאוד
    n, m = cost_matrix.shape
    max_dim = max(n, m)
    if n < max_dim:
        cost_matrix = np.vstack([cost_matrix, np.full((max_dim - n, m), 1e6)])
    if m < max_dim:
        cost_matrix = np.hstack([cost_matrix, np.full((max_dim, max_dim - m), 1e6)])

    # ביצוע האלגוריתם ההונגרי
    row_ind, col_ind = linear_sum_assignment(cost_matrix)

    # קבלת התוצאות
    for row, col in zip(row_ind, col_ind):
        if row < len(call_df) and col < len(vol_df):
            distance = geodesic(call_df.iloc[row]['מיקום תקלה'], vol_df.iloc[col]['מיקום כונן']).kilometers
            assignment = (
                call_df.iloc[row]['מספר קריאה'],  # מזהה קריאה
                call_df.iloc[row]['מזהה פונה'],  # מזהה פונה
                vol_df.iloc[col]['מזהה כונן'],   # מזהה כונן
                category,                        # קטגוריה
                distance,                        # מרחק בק"מ
                cost_matrix[row, col]            # עלות
            )
            results.append(assignment)

# יצירת DataFrame עם העמודות בסדר הרצוי
results_df = pd.DataFrame(results, columns=['מספר קריאה', 'מזהה פונה', 'מזהה כונן', 'קטגוריה', 'מרחק בק"מ', 'עלות'])

# הדפסת התוצאות
results_df


,מספר קריאה,מזהה פונה,מזהה כונן,קטגוריה,"מרחק בק""מ",עלות
0,82329,0170 יעקב,1110 יוסף,הנעה,5.764942,26.983313
1,73636,0900 אבי,6737 משה,הנעה,6.800213,28.395030
2,72143,1712 ניסים,4337 יזהר,הנעה,31.722779,46.691580
3,40496,5375 ששון,2045 מיכי,הנעה,23.964871,32.583941
4,50306,0124 אלעזר,2972 יוני,הנעה,16.555543,16.555543
5,92522,3753 ליאור,0783 משה,הנעה,7.951755,29.938946
6,34699,0019 משה,1350 יעקב,הנעה,22.404145,40.381491
7,61299,8740 אלי,3701 נתנאל,הנעה,10.055904,32.479049
8,55257,2142 משה,6474 ירמיהו,הנעה,12.151232,39.794278
9,13729,5574 מחמוד,2632 בצלאל,הנעה,26.972752,34.359544


In [12]:
results_df.to_csv('results.csv', index=False)

# בניית ממשק

In [43]:
# import streamlit as st
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from scipy.optimize import linear_sum_assignment
import requests

# טוען את קובץ ה-CSV מ-GitHub
url = 'https://raw.githubusercontent.com/Celinmi/Final-Project/main/volunteer_pool.csv'
Volunteer_pool = pd.read_csv(url)
logo_url = 'https://raw.githubusercontent.com/Celinmi/Final-Project/main/ידידים - לוגו.jpg'
back_ground_url = 'https://raw.githubusercontent.com/Celinmi/Final-Project/main/ידידים - רקע.jpg'
categories = ['הנעה', "פנצ'ר", 'רכב נעול', 'קורונה - קניית תרופות', 'דלת', 'אחר', 'שמן-מים-דלק', 'שינוע', 'קורונה - שינוע מזון', 'קורונה - קניית אוכל', 'חילוץ שטח']

st.markdown(
    f"""
    <style>
    .main {{
        background-image: url("{back_ground_url}");
        background-size: 90vw 100vh;  # This sets the size to cover 100% of the viewport width and height
        background-position: center;
        background-repeat: no-repeat;
    }}
    .stButton button {{
        background-color: rgba(255, 255, 255, 0.8);
        color: black;
        border-radius: 12px;
    }}
    .stTextInput, .stNumberInput input {{
        color: black;
    }}
    .title {{
        background-color: rgba(255, 255, 255, 0.8); /* רקע בהיר */
        color: black !important;
        padding: 10px;
        border-radius: 10px;
    }}
    .header {{
        background-color: rgba(255, 255, 255, 0.8); /* רקע בהיר */
        color: black !important;
        padding: 5px;
        border-radius: 5px;
    }}
    .css-10trblm, .css-1v3fvcr p {{
        color: white !important;
    }}
    .stTitle, .stHeader, .stSubheader, .stMarkdown, .stText, .stNumberInput label, .stDateInput label {{
        color: white !important;
    }}
    </style>
    """,
    unsafe_allow_html=True
)

st.image(logo_url, width=200)

st.markdown('<h1 class="title">"בחירת מתנדב אופטימלי"</h1>', unsafe_allow_html=True)

st.markdown('<h2 class="header">הזן פרטים לבחירת כונן</h2>', unsafe_allow_html=True)

# קלטים מהמשתמש
location = st.text_input("הכנס מיקום תקלה - קו אורך, קו רוחב לדוגמה 34.784 , 32.085")
caller_id = st.text_input("הכנס מזהה ייחודי של הפונה - שם פרטי ו 4 ספרות אחרונות של ת.ז")
category = st.selectbox("בחר קטגוריה של התקלה", categories)

if st.button("בחר מתנדב"):
    if location and caller_id:
        Volunteer_pool = Volunteer_pool[Volunteer_pool['זמינות'] == 'זמין']

        # הכנת פול של פניות
        calling_pool = pd.DataFrame({
            'מזהה פונה': [caller_id],
            'קטגוריה': [category],
            'מיקום תקלה': [location]
        })

        # הכנת מטריצת העלויות
        cost_matrix = []
        for call_index, call_row in calling_pool.iterrows():
            row = []
            call_location = tuple(map(float, call_row['מיקום תקלה'].split(',')))
            for vol_index, vol_row in Volunteer_pool.iterrows():
                vol_location = tuple(map(float, vol_row['מיקום כונן'].split(',')))
                distance = geodesic(call_location, vol_location).kilometers
                row.append(distance)
            cost_matrix.append(row)

        cost_matrix = np.array(cost_matrix)

        # נרמול ציוני המתנדבים
        def normalize_scores(scores):
            return (scores - scores.min()) / (scores.max() - scores.min())

        volunteer_scores = normalize_scores(Volunteer_pool['final score'])

        # הוספת הציונים למטריצת העלויות
        cost_matrix += volunteer_scores.values.reshape(1, -1)

        # וידוא שהמטריצה ריבועית
        n, m = cost_matrix.shape
        max_dim = max(n, m)
        if n < max_dim:
            cost_matrix = np.vstack([cost_matrix, np.full((max_dim - n, m), 1e6)])
        if m < max_dim:
            cost_matrix = np.hstack([cost_matrix, np.full((max_dim, max_dim - m), 1e6)])

        # ביצוע האלגוריתם ההונגרי
        row_ind, col_ind = linear_sum_assignment(cost_matrix)

        # קבלת התוצאות
        results = []
        for row, col in zip(row_ind, col_ind):
            if row < len(calling_pool) and col < len(Volunteer_pool):
                distance = geodesic(calling_pool.iloc[row]['מיקום תקלה'], Volunteer_pool.iloc[col]['מיקום כונן']).kilometers
                assignment = (
                    calling_pool.iloc[row]['מזהה פונה'],
                    Volunteer_pool.iloc[col]['מזהה כונן'],
                    category,
                    distance
                )
                results.append(assignment)

        results_df = pd.DataFrame(results, columns=['מזהה פונה', 'מזהה כונן', 'קטגוריה', 'מרחק בק"מ'])

        # הצגת התוצאות
        st.write("תוצאות הבחירה:")
        st.dataframe(results_df)

    else:
        st.write("יש להזין את כל השדות.")

תוצאות הבחירה:


,מזהה פונה,מזהה כונן,קטגוריה,"מרחק בק""מ"
0,12345,0220 יוסף,הנעה,1.294065


In [ ]:
# requirements.txt

# pandas == 2.1.4
# numpy == 1.26.4
# geopy == 2.4.1
# scipy == 1.13.1
# requests == 2.32.3
